In [1]:
import geoplot
import geoplot.crs as gcrs
import pandas as pd
import folium
import geopandas as gpd
from ipywidgets import interact
import numpy as np
from folium import CircleMarker
from folium.plugins import HeatMap
import branca

In [3]:
# Charger les données CSV
df = pd.read_csv('data/transport_traffic_voiture.csv', sep=';')

# Assurer que les années sont en chaînes de caractères
years = [str(year) for year in range(1993, 2024)]  # Génère la liste des années comme chaînes de caractères
df.columns = ['LA Code', 'Local Authority'] + years  # S'assurer que les colonnes sont nommées correctement

# Charger le fichier GeoJSON des boroughs de Londres
gdf = gpd.read_file('data/london_boroughs.geojson')

# Fonction pour générer une échelle de couleurs plus détaillée
def generate_color_scale(values):
    # Calculer la plage des valeurs
    min_value = np.min(values)
    max_value = np.max(values)
    
    # Créer une échelle de couleurs avec plus de nuances
    color_scale = branca.colormap.linear.RdYlBu_09.scale(min_value, max_value)
    color_scale = color_scale.to_step(n=20)  # Crée une échelle avec plus de nuances (20 étapes)
    
    return color_scale

def update_map(year):
    # Vérifier si l'année sélectionnée existe dans les colonnes
    if str(year) not in df.columns:
        print(f"Année {year} introuvable dans les colonnes !")
        return
    
    # Créer une carte centrée sur Londres (réinitialisation)
    m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)
    
    # Récupérer toutes les valeurs de l'année sélectionnée
    values = df[str(year)].values[2:]  # Les valeurs commencent à l'index 2, car les deux premières colonnes ne sont pas des années
    
    # Générer l'échelle de couleurs détaillée
    color_scale = generate_color_scale(values)
    
    # Ajouter les polygones des boroughs avec les couleurs en fonction des valeurs
    for _, row in gdf.iterrows():
        borough_name = row['name']  # Nom du borough
        if borough_name in df['Local Authority'].values:
            # Extraire la géométrie du borough
            geometry = row['geometry']
            
            # Trouver la valeur de l'année pour le borough
            value = df.loc[df['Local Authority'] == borough_name, str(year)].values[0]
            
            # Vérifier si la valeur n'est pas manquante ou incorrecte
            if pd.isna(value) or value < 0:
                value = 0  # Fixer une valeur par défaut (ou choisir de l'ignorer)
            
            # Obtenir la couleur en fonction de la valeur
            color = color_scale(value)  # Utiliser l'échelle de couleurs dynamique pour obtenir la couleur
            
            # Créer le texte du popup avec les informations à afficher
            popup_text = f"<b>{borough_name}</b><br>Valeur en {year}: {value}"
            
            # Ajouter le polygone du borough à la carte avec la couleur correspondante
            folium.GeoJson(
                geometry,
                style_function=lambda feature, color=color: {
                    'fillColor': color,  # Remplissage du polygone avec la couleur associée à la valeur
                    'color': 'darkgrey',  # Bordure des polygones
                    'weight': 0.3,  # Épaisseur de la bordure
                    'fillOpacity': 0.6  # Opacité du remplissage
                },
                tooltip=popup_text  # Afficher les informations au survol
            ).add_to(m)
    
    # Ajouter la légende de l'échelle de couleur
    color_scale.caption = "Valeurs par borough (échelle dynamique)"
    color_scale.add_to(m)
    
    # Afficher la carte
    return m

# Créer un dropdown pour sélectionner l'année
interact(update_map, year=[str(year) for year in range(1993, 2024)]);  # Plage d'années entre 1993 et 2023

interactive(children=(Dropdown(description='year', options=('1993', '1994', '1995', '1996', '1997', '1998', '1…

In [36]:
# Sauvegarder la carte pour une année spécifique (par exemple, 2023)
m = update_map(1993)
m.save('static/traffic_map_1993.html')

In [4]:
from ipywidgets import interact, IntSlider, fixed
import folium
import pandas as pd
import geopandas as gpd
import branca
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Ta fonction pour le clustering et l'update de la carte ici
def cluster_boroughs(df, year, n_clusters=3):
    year = str(year)
    
    if year not in df.columns:
        print(f"L'année {year} est introuvable dans les colonnes!")
        return df
    
    borough_values = df[year].values
    borough_values_reshaped = borough_values.reshape(-1, 1)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(borough_values_reshaped)
    
    df['Cluster'] = kmeans.labels_
    
    return df, kmeans

def update_map_with_clusters(df, gdf, year):
    df_clustered, _ = cluster_boroughs(df, year, n_clusters=3)
    
    m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)
    
    color_scale = branca.colormap.linear.Paired_12.scale(0, 2)
    
    for _, row in gdf.iterrows():
        borough_name = row['name']
        if borough_name in df['Local Authority'].values:
            geometry = row['geometry']
            cluster_label = df.loc[df['Local Authority'] == borough_name, 'Cluster'].values[0]
            color = color_scale(cluster_label)
            
            popup_text = f"<b>{borough_name}</b><br>Cluster: {cluster_label}"
            
            folium.GeoJson(
                geometry,
                style_function=lambda feature, color=color: {
                    'fillColor': color,
                    'color': 'darkgrey',
                    'weight': 0.3,
                    'fillOpacity': 0.6
                },
                tooltip=popup_text
            ).add_to(m)
    
    color_scale.caption = "Clusters des boroughs"
    color_scale.add_to(m)
    
    return m

# Exemple de DataFrame et GeoDataFrame pour tests
df = pd.read_csv('data/transport_traffic_voiture.csv', sep=';')
gdf = gpd.read_file('data/london_boroughs.geojson')

# Créer un widget interactif pour l'année
interact(update_map_with_clusters, df=fixed(df), gdf=fixed(gdf), year=IntSlider(min=1993, max=2023, step=1, value=2023));


interactive(children=(IntSlider(value=2023, description='year', max=2023, min=1993), Output()), _dom_classes=(…

In [ ]:
import os

def update_map_with_clusters(df, gdf, year):
    df_clustered, _ = cluster_boroughs(df, year, n_clusters=3)

    m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

    color_scale = branca.colormap.linear.Paired_12.scale(0, 2)

    for _, row in gdf.iterrows():
        borough_name = row['name']
        if borough_name in df['Local Authority'].values:
            geometry = row['geometry']
            cluster_label = df.loc[df['Local Authority'] == borough_name, 'Cluster'].values[0]
            color = color_scale(cluster_label)

            popup_text = f"<b>{borough_name}</b><br>Cluster: {cluster_label}"

            folium.GeoJson(
                geometry,
                style_function=lambda feature, color=color: {
                    'fillColor': color,
                    'color': 'darkgrey',
                    'weight': 0.3,
                    'fillOpacity': 0.6
                },
                tooltip=popup_text
            ).add_to(m)

    color_scale.caption = "Clusters des boroughs"
    color_scale.add_to(m)

    # Sauvegarder la carte dans un fichier HTML
    output_dir = 'static/traffic_pred'
    os.makedirs(output_dir, exist_ok=True)
    map_file_path = os.path.join(output_dir, f'traffic_map_pred_{year}.html')
    m.save(map_file_path)

    return m

# Exemple de DataFrame et GeoDataFrame pour tests
df = pd.read_csv('data/transport_traffic_voiture.csv', sep=';')
gdf = gpd.read_file('data/london_boroughs.geojson')

# Créer un widget interactif pour l'année
interact(update_map_with_clusters, df=fixed(df), gdf=fixed(gdf), year=IntSlider(min=1993, max=2023, step=1, value=2023));

interactive(children=(IntSlider(value=2023, description='year', max=2023, min=1993), Output()), _dom_classes=(…